# Topic 38: Time Series Models

https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/

* AR model (autoregression)
* MA model (moving average)
* ARMA model (autoregression + moving average)
* Differencing model
* ARIMA model (autoregression + differencing + moving average)
* SARIMA model (seasonal ARIMA)
* ARIMAX model (ARIMA + exogenous variables)
* SARIMAX model (seasonal ARIMA + exogenous variables)

## Auto-regressive Time Series Model

An autoregression model makes an assumption that the observations at previous time steps are useful to predict the value at the next time step. It is one of the simplest time series models in which we use a linear model to predict the value at the present time using the value at the previous time. 

<p style='text-align:center; font-size: 30px;'>𝑌<sub>t</sub>=𝜙<sub>1</sub>𝑌<sub>𝑡−1</sub>+𝜖<sub>𝑡</sub></p>

The numeral one (1) denotes that the next instance is solely dependent on the previous instance.  The 𝜙(phi) is a coefficient which we seek so as to minimize the error function.

The order of AR is the number of lag terms we are using to predict the present value (AR(1) uses only 1 lag - one value directly preceding the value you are trying to predict, AR(2) use the two values directly preceding the value you are trying to predict) 

#### How do we determine the order aka how many lag terms do we include? 

Using ACF and PACF! 

<img src='resources/AR(1).png'>

### AR(1) Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.arima.model import ARIMA

#read in csv
shampoo = pd.read_csv('resources/shampoo.csv', header=0, usecols=[1]).iloc[:-1]
#plot of data to see visualize trends
shampoo.plot()

In [ ]:
# train-test-split for time series isn't random!
train, test = shampoo.iloc[:24], shampoo.iloc[24:]

In [ ]:
# order = (p, d, q)
# p - autoregressive
# d - differences
# q - moving average

ar1 = ARIMA(train, order=(1,0,0)).fit()
trainpreds = ar1.predict()
testpreds = ar1.forecast(12)

In [ ]:
ar1.summary()

In [ ]:
plt.plot(shampoo, label='shampoo')
plt.plot(trainpreds.append(testpreds), label='AR=1')
plt.legend()
plt.show()

In [ ]:
print('Train RMSE: ', mean_squared_error(train, trainpreds)**0.5)
print('Test RMSE: ', mean_squared_error(test, testpreds)**0.5)

### AR(2) Model

In [ ]:
ar2 = ARIMA(train, order=(2,0,0)).fit()
ar2preds = ar2.predict()
ar2.summary()

In [ ]:
plt.plot(shampoo, label='shampoo')
plt.plot(ar2preds.append(ar2.forecast(12)), label='AR=2 with 10 day forecast')
plt.legend()
plt.show()

In [ ]:
print('Train RMSE: ', mean_squared_error(train, ar2.predict())**0.5)
print('Test RMSE: ', mean_squared_error(test, ar2.forecast(12))**0.5)

## Moving Average Time Series Model

Sometimes, a past value is not a useful indicator of what value will come next. Consider a system that is subject to a lot of shocks/volatility. If a previous time period experiences a shock it may cause an error for future predictions if we just that value. A moving average model helps address this behavior. 

A moving average term in a time series model is a past error (multiplied by a coefficient).

An MA model assumes present value is related to errors in the past - includes memory of past errors


<p style='text-align: center; font-size:30px;'>𝑌<sub>t</sub>=μ + 𝜖<sub>𝑡</sub>+𝜃<sub>1</sub>𝜖<sub>𝑡−1</sub></p>

For more details on how this model is fit: https://stats.stackexchange.com/questions/26024/moving-average-model-error-terms/74826#74826 

## Differencing Model aka Integrated Model

The differenced value is equal to the present value minus the value at the next lag. A time series which needs to be differenced to be made stationary is said to be an "integrated" time series.

<p>If d=0:  y<sub>t</sub>  =  Y<sub>t</sub></p>

If d=1:  y<sub>t</sub> =  Y<sub>t</sub> - Y<sub>t-1</sub>

If d=2:  y<sub>t</sub> =  (Y<sub>t</sub> - Y<sub>t-1</sub>) - (Y<sub>t-1</sub> - Y<sub>t-2</sub>)  =  Y<sub>t</sub> - 2Y<sub>t-1</sub> + Y<sub>t-2</sub>

In [ ]:
#get differenced values
diff = shampoo.diff()
diff2 = diff.diff()

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))
ax1.plot(shampoo, label='Shampoo')
ax2.plot(diff, label='Shampoo, I=1')
ax3.plot(diff2, label='Shampoo, I=2')
fig.legend()
fig.show()

## ARIMA

Combines AR, Differencing (I), and MA

The differenced value is equal to the present value minus the value at the next lag. A time series which needs to be differenced to be made stationary is said to be an "integrated" time series.

<p style ='text-align:center; font-size: 30px;'>𝑌<sub>t</sub>=𝜙<sub>1</sub>𝑌<sub>𝑡−1</sub>+𝜙<sub>2</sub>𝑌<sub>𝑡−2</sub>...𝜙<sub>𝑝</sub>𝑌<sub>t−𝑝</sub>+𝜖<sub>𝑡</sub>+𝜃<sub>1</sub>𝜖<sub>𝑡−1</sub>+𝜃<sub>2</sub>𝜖<sub>𝑡−2</sub>+...𝜃<sub>𝑞</sub>𝜖<sub>𝑡−𝑞</sub></p>



ARIMA has three main parameters we need to input, p, d, & q

<b>p:</b> The number of AR terms we are going to include<br/>
<b>d:</b> The number of times we are differencing our data<br/>
<b>q:</b> The number MA terms we are going to include

In [ ]:
#perform dickey fuller to see if our data is stationary

from statsmodels.tsa.stattools import adfuller
dickeyfuller = adfuller(shampoo.values)
pd.DataFrame(dickeyfuller[0:4], index=['Test Statistic','p-value','# Lags Used','Number of Observations Used'])
# dickeyfuller

In [ ]:
dickeyfuller_differenced

In [ ]:
dickeyfuller_differenced = adfuller(shampoo.diff()[1:])
pd.DataFrame(dickeyfuller_differenced[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])


In [ ]:
#ACF/PACF to determine which terms in include (MA or AR or Both?)
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

#plot autocorrelation for each lag (alpha is confidence interval)
plot_acf(shampoo, alpha=.05)
plt.show()

In [ ]:
plot_pacf(shampoo, alpha=.05)
plt.show()

## ARIMA implementation

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
#fit ARIMA model (2,1,1))

arima1 = ARIMA(train, order=(2,1,1)).fit()
trainpreds = np.append(np.array([0, 0]), np.array(arima1.predict()))
testpreds = arima1.forecast(12)[0]
arima1.summary()

In [ ]:
trainpreds

In [ ]:
testpreds

In [ ]:
plt.plot(shampoo, label='shampoo')
plt.plot(np.append(trainpreds, testpreds), label='ARIMA (2, 1, 0) with 10 day forecast')
plt.legend()
plt.show()

In [ ]:
print('Train RMSE: ', mean_squared_error(train, trainpreds)**0.5)
print('Test RMSE: ', mean_squared_error(test, testpreds)**0.5)

## ARIMAX

ARIMA with eXogenous variables - extend ARIMA to include additional variables that might have an impact on what we are are trying to forecast. 

Considerations: 

1) Does our exogenous variable actually impact our endogenous variable (and not the other way around - use granger causality test) 

2) Exogenous variables need to be differenced at the same order as the endogenous 

<img src='resources/seasonal_data.png'/>

### Steps to build a (S)ARIMA(X) Model

https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html

1. Plot data, see if there are trends <br/>
2. If trends, remove them (differencing, log transform, etc) <br/>
3. If seasonal trends are there determine periodicity. <br/>
4. ACF and PACF of  data <br/>
5. Determine order of differencing, AR, or MA (or both) <br/>
6. Build model and evaluate 


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# SARIMAX(shampoo, exog=None, seasonal_order=(0, 0, 0, 52))

# Additional Resources

* Modeling cheat sheet: https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/
* AutoARIMA: https://towardsdatascience.com/time-series-forecasting-using-auto-arima-in-python-bb83e49210cd
* SARIMAX Project walkthrough: https://towardsdatascience.com/newyork-taxi-demand-forecasting-with-sarimax-using-weather-data-d46c041f3f9c